### Make separate snakefile to process viruses
- After atlas already done: contigs present. Can run virsorter2 on these
- Need to make a snakefile for this
- 236 fa files still need to go through vs2 (is ok)

Just make 2: One for running virsorter, then one for moving the files. And renaming.
Viral taxonomy? With sourmash?

Do we want to use genomad instead? bc plasmids. 

In [ ]:
genomad end-to-end \
--cleanup --splits \
8 GCF_009025895.1.fna.gz \
genomad_output genomad_db

In [ ]:
#GENOMAD
# srun it, needs quite some mem
srun --account=ctbrowngrp -p med2 -J genomad -t 12:00:00 -c 36 --mem=100gb --pty bash

# end to end for everything, need to annotate for classification
# Use for taxonomy of DNA phage
# softlink db
ln -s /group/jbemersogrp/databases/genomad/genomad_db .

# own genomad db
/home/amhorst/databases/genomad/genomad_db

# running 
srun --account=ctbrowngrp -p high2 -J gmd -t 24:00:00 -c 120 --mem=100gb --pty bash
mamba activate branchwater-skipmer

snakemake \
-s genomad.smk \
--rerun-triggers mtime --rerun-incomplete \
--resources mem_mb=100000 -c 120 --use-conda \
-k -n


In [ ]:
# running atlas in a screen, still wil submit jobs to cluster
srun --account=ctbrowngrp -p med2 -J vs2 -t 48:00:00 -c 36 --mem=50gb --pty bash
mamba activate branchwater-skipmer

snakemake \
-s virsorter2.smk \
--rerun-triggers mtime --rerun-incomplete \
--resources mem_mb=50000 -c 36 --use-conda \
-k -n


In [ ]:
# another snakefile for renaming and using the quality tresh. 

# rename found vOTUs
rule bbmap_rename_viral:
    input: 
        check = "../results/check/{sample}_vs2_done.check"
    output: 
        contigs = "../results/virsorter2/contigs/{sample}_rename.fa",
        check = "../results/check/{sample}_rename.check"
    log:
        "../results/logs/virsorter2/{sample}_rename.log"
    conda: 
        "bbmap"
    shell:
        """
    if [[ ! -e ../results/virsorter2/{wildcards.sample}/final-viral-combined.fa ]] 
    then
        touch {output.contigs} 
    else
        rename.sh in=../results/virsorter2/{wildcards.sample}/final-viral-combined.fa \
        out={output.contigs} prefix=AtH2023_{wildcards.sample}_viral && touch {output.check}
    fi
        """


